<a href="https://colab.research.google.com/github/dannezri/script_planning_check/blob/main/planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import json
import requests
import os


# URL pour déclencher l'extraction du planning avec la date sélectionnée
extraction_url = f"https://planning.geniusclass.fr/request/extraction_planning.php?date_debut=2025-02-01&date_fin=2025-02-02"
json_url = "https://planning.geniusclass.fr/request/templates/combined.json"

# Déclencher la génération du JSON
print("🔄 Extraction du planning en cours...")
requests.get(extraction_url)

# Télécharger le fichier JSON
print("🔽 Téléchargement du fichier JSON...")
response = requests.get(json_url)
if response.status_code == 200:
    file_path = "combined.json"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(response.text)
    print("✅ Fichier JSON téléchargé avec succès.")
else:
    print("❌ Erreur lors du téléchargement du fichier JSON.")
    exit()

# Charger le JSON
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

data = load_json(file_path)

# Fonction de validation du planning
def validate_schedule(data):
    errors = []
    tuteur_activites = {}  # Dictionnaire pour stocker les affectations par tuteur et par jour

    for centre in data["centres"]:
        centre_id = centre["idCentre"]

        for date, schedule in centre["dates"].items():
            for heure, session in schedule["heures"].items():
                tuteurs = session.get("tuteurs", [])

                if len(tuteurs) > 3:
                    errors.append(f"🚨 Centre {centre_id}, {date} {heure}: Plus de 3 groupes simultanés")

                for tuteur in tuteurs:
                    tuteur_id = tuteur["idTuteur"]
                    tuteur_prenom = tuteur["prenom"]
                    tuteur_nom = tuteur["nom"]

                    if (tuteur_id, date) not in tuteur_activites:
                        tuteur_activites[(tuteur_id, date)] = []

                    tuteur_activites[(tuteur_id, date)].append((heure, centre_id))

                    if "matieres_dispensees" not in tuteur or not tuteur["matieres_dispensees"]:
                        errors.append(
                            f"⚠️ Centre {centre_id}, {date} {heure}: Tuteur {tuteur_id} ({tuteur_prenom} {tuteur_nom}) n'a pas de matières définies."
                        )
                        continue

                    matieres_dispensees = {m["matiere"]: int(m["niveau"]) for m in tuteur["matieres_dispensees"]}

                    for eleve in tuteur["eleves"]:
                        eleve_id = eleve["idEleve"]
                        eleve_prenom = eleve["prenom"]
                        eleve_nom = eleve["nom"]
                        eleve_niveau = eleve["niveauEleve"]
                        eleve_matieres = eleve["matieres"]

                        matieres_verifiees = []
                        for matiere in eleve_matieres:
                            if "Aide aux devoirs" in matiere:
                                matieres_verifiees.extend(["Maths", "Français"])
                            else:
                                matieres_verifiees.append(matiere)

                        for matiere in matieres_verifiees:
                            if matiere not in matieres_dispensees:
                                errors.append(
                                    f"❌ Centre {centre_id}, {date} {heure}: Élève {eleve_id} ({eleve_prenom} {eleve_nom}) suit {matiere} avec le tuteur {tuteur_prenom}, qui ne l'enseigne pas."
                                )
                            elif matieres_dispensees[matiere] < eleve_niveau:
                                errors.append(
                                    f"❌ Centre {centre_id}, {date} {heure}: Élève {eleve_id} ({eleve_prenom} {eleve_nom}) niveau {eleve_niveau} en {matiere}, mais le tuteur {tuteur_prenom} n'a qu'un niveau {matieres_dispensees[matiere]}."
                                )

    for (tuteur_id, date), activites in tuteur_activites.items():
        unique_centres = set(centre_id for _, centre_id in activites)
        if len(unique_centres) > 2:
            erreurs_centres = ", ".join([f"Centre {centre} à {heure}" for heure, centre in activites])
            errors.append(
                f"🚨 Tuteur {tuteur_id} affecté à plus de 2 centres le {date}. Détails: {erreurs_centres}"
            )

    return errors

# Exécution des validations
errors_list = validate_schedule(data)

# Sauvegarde des erreurs dans un fichier JSON
# Définir un dossier de sortie
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)  # Crée le dossier s'il n'existe pas

errors_file = os.path.join(output_dir, "errors.json")

# Générer un exemple de résultat
result_file = os.path.join(output_dir, "results.json")
results = {"status": "Succès", "details": "Le script s'est bien exécuté"}

# Sauvegarde des résultats dans un fichier JSON
with open(result_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4)

print("✅ Résultats sauvegardés :", result_file)


# Sauvegarde des erreurs dans un fichier JSON
with open(errors_file, "w", encoding="utf-8") as f:
    json.dump({"errors": errors_list}, f, indent=4)

print("✅ Fichier des erreurs sauvegardé :", errors_file)

with open(errors_file, "w", encoding="utf-8") as f:
    json.dump({"errors": errors_list}, f, indent=4)

# Affichage des erreurs
if errors_list:
    print("\n🚨 Erreurs détectées dans le planning :")
    for error in errors_list[:50]:
        print(f"- {error}")
    print(f"\n🔹 {len(errors_list)} erreurs trouvées. Voir le fichier : {errors_file}")
else:
    print("\n✅ Aucun problème détecté dans le planning.")


🔄 Extraction du planning en cours...
🔽 Téléchargement du fichier JSON...
✅ Fichier JSON téléchargé avec succès.
✅ Résultats sauvegardés : outputs/results.json
✅ Fichier des erreurs sauvegardé : outputs/errors.json

🚨 Erreurs détectées dans le planning :
- ❌ Centre 0, 2025-02-01 11:00:00: Élève 887 (Amine  Hafidhi) suit Physique avec le tuteur Haansika, qui ne l'enseigne pas.
- ❌ Centre 0, 2025-02-01 11:00:00: Élève 1039 (Rodrigue Longu) suit SVT avec le tuteur Ajin, qui ne l'enseigne pas.
- ❌ Centre 0, 2025-02-02 13:30:00: Élève 1065 (Aisha Mirza) niveau 11 en Français, mais le tuteur Sharusan n'a qu'un niveau 10.
- ❌ Centre 0, 2025-02-02 11:00:00: Élève 1039 (Rodrigue Longu) suit SVT avec le tuteur Ajin, qui ne l'enseigne pas.
- ❌ Centre 0, 2025-02-02 11:00:00: Élève 1063 (Aylan Aouadene) niveau 10 en Français, mais le tuteur Haansika n'a qu'un niveau 8.
- ❌ Centre 1, 2025-02-01 09:30:00: Élève 665 (Ethan Irelus) niveau 8 en Français, mais le tuteur Haris n'a qu'un niveau 4.
- ❌ Centr